# 1) Extract CAD data to GIS

### a) Create a new FGDB and set it as the workspace

In [ ]:
##~~~~~~~~~~~ Create a scratch GDB to house data ~~~~~~~~~~~##
out_gdb_name = "CAD2GIS.gdb"

arcpy.management.CreateFileGDB(out_folder_path, out_gdb_name)

arcpy.AddMessage("Scratch GDB created...")

# set scratch GDB
scratch_gdb = f"{out_folder_path}/{out_gdb_name}"

# set workspace
arcpy.env.workspace = scratch_gdb

arcpy.AddMessage("Workspace set to scratch GDB...")

### b) Extract CAD data

In [ ]:
##~~~~~~~~~~~ CAD to GIS (feature dataset creation) ~~~~~~~~~~~##

# set variables for CADtoGDB tool
out_gdb_path = scratch_gdb
out_dataset_name = "CAD2GIS"
reference_scale = "1000"

arcpy.conversion.CADToGeodatabase(input_cad_dataset, out_gdb_path,
                                  out_dataset_name, reference_scale,
                                  spatial_ref)

arcpy.AddMessage("CAD to GIS complete...")

### c) Standardize field values

In [ ]:
##~~~~~~~~~~~ recalculate "BUILD_STATUS", "INSTALLATION_YEAR", "TDS_WBS_CODE", "MARKET_CODE" field values in "Point" FC ~~~~~~~~~~~##
point_fc = "Point"
field_name = "BUILD_STATUS"

# BUILD STATUS
# Open an update cursor on the feature class
with arcpy.da.UpdateCursor(point_fc, field_name) as cursor:
    for row in cursor:
        # Calculate the value for the field
        value = "As Staked"
        # Update the field value for all rows
        row[0] = value
        cursor.updateRow(row)

# INSTALLATION YEAR
field_name = "INSTALLATION_YEAR"

# Open an update cursor on the feature class
with arcpy.da.UpdateCursor(point_fc, field_name) as cursor:
    for row in cursor:
        # Calculate the value for the field
        value = "2023"
        # Update the field value for all rows
        row[0] = value
        cursor.updateRow(row)

# TBS_WBS_CODE
field_name = "TDS_WBS_CODE"

# Open an update cursor on the feature class
with arcpy.da.UpdateCursor(point_fc, field_name) as cursor:
    for row in cursor:
        # Calculate the value for the field
        value = proj_wbs_code
        # Update the field value for all rows
        row[0] = value
        cursor.updateRow(row)

# MARKET CODE
field_name = "MARKET_CODE"

# Open an update cursor on the feature class
with arcpy.da.UpdateCursor(point_fc, field_name) as cursor:
    for row in cursor:
        # Calculate the value for the field
        value = market_code
        # Update the field value for all rows
        row[0] = value
        cursor.updateRow(row)

arcpy.AddMessage("Point fields calculated...")

##### *** this process gets repeated for the "Polyline" and "Polygon" FC's as well ***

### d) Polyline specific field standardization

##### For features that meet the where clause (SQL query), the code first copies and pastes the values in the "Layer" field to the "COMMENTS" field, then updates the "Layer" field value to == "Fiber Cable"

In [ ]:
# Move "Layer" field values to "COMMENT" field and change "Layer" value to == "Fiber Cable"... for rows that meet the where_clause variable!
polyline_fc = "Polyline"
layer_field = "Layer"
comments_field = "COMMENTS"
where_clause = "Entity = 'LWPolyline' And Layer LIKE '%Count%'"

with arcpy.da.UpdateCursor(polyline_fc, [layer_field, comments_field], where_clause) as cursor:
    for row in cursor:
        cursor.updateRow(["Fiber Cable", row[0]])

arcpy.AddMessage('"Layer" and "COMMENT" field values updated...')

### f) Add Feature Classes (FCs) to the Table of Contents (TOC)

In [ ]:
# add "Polygon" FC to the bottom of TOC
polygon_path = f"{scratch_gdb}/{out_dataset_name}/{polygon_fc}"
m.addDataFromPath(polygon_path)

# add "Polyline" FC to the TOC
polyline_path = f"{scratch_gdb}/{out_dataset_name}/{polyline_fc}"
m.addDataFromPath(polyline_path)

# add "Point" FC to the top of TOC
point_path = f"{scratch_gdb}/{out_dataset_name}/{point_fc}"
m.addDataFromPath(point_path)

arcpy.AddMessage('"Point", "Polyline", and "Polygon" FCs added to TOC...')

# 2) Translate features to TDS schema

### a) set workspace

##### You could hardcode these variables if you wanted... but since it's a script tool "arcpy.GetParameterAsText()" is used so that the user can specify the desired GDB and spatial reference

In [ ]:
# set project specific variables
scratch_gdb = arcpy.GetParameterAsText (0)
spatial_ref = arcpy.GetParameterAsText (1)

# set workspace
arcpy.env.workspace = scratch_gdb

### b) Polyline management

##### The goal here is to get data from the "Point" FC into the "Polyline" FC... this was the work around that I came up with but there's likely a more efficient way to do this...

#### - create a copy of the "Point" FC that was created in step 1b

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~ create copy of "Point" FC ~~~~~~~~~~~~~~~~~~~~~~~~~#
copy_name = "InputPointCopy"
out_copy = f"{scratch_gdb}\{copy_name}"

arcpy.conversion.ExportFeatures(point_fc, copy_name)

arcpy.AddMessage("'InputPointCopy' FC created...")

#### - join the copied Point FC ("InputPointCopy") with the "Polyline" FC

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~ join "InputPointCopy" FC table with "Polyline" FC table ~~~~~~~~~~~~~~~~~~~~~~~~~#
inLayer = polyline_fc
inField = "Handle"
joinTable = copy_name
joinField = "TARGET_HANDLE"
joinType = "KEEP_ALL"
index_join_fields = "NO_INDEX_JOIN_FIELDS"

arcpy.management.AddJoin(inLayer,
                         inField,
                         joinTable,
                         joinField,
                         joinType,
                         index_join_fields)

arcpy.AddMessage("Join complete...")

#### - create a copy of the joined "Polyline" FC so that the fields are editable

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~ create copy of "Polyline" FC so that fields are editable ~~~~~~~~~~~~~~~~~~~~~~~~~#
line_copy = "Polyline_join"
line_copy_path = f"{scratch_gdb}\{line_copy}"

arcpy.management.CopyFeatures(polyline_fc, line_copy)
arcpy.AddMessage(f'"{line_copy}" FC created...')

#### - copy values from fields starting with "InputPointCopy_" to fields starting with "Polyline_"

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~ copy values from "InputPointCopy_" fields to "Polyline_" fields ~~~~~~~~~~~~~~~~~~~~~~~~~#

# create tuble of field names [("take values from this field", "and put them in this field")]
field_pairs = [
    ("InputPointCopy_FIBER_CABLE_TYPE", "Polyline_FIBER_CABLE_TYPE"),
    ("InputPointCopy_FIBER_CABLE_NAME", "Polyline_FIBER_CABLE_NAME"),
    ("InputPointCopy_ROUTE", "Polyline_ROUTE"),
    ("InputPointCopy_PLACEMENT_TYPE", "Polyline_PLACEMENT_TYPE"),
    ("InputPointCopy_CABLE_SEQUENTIAL_LENGTH", "Polyline_CABLE_SEQUENTIAL_LENGTH"),
    ("InputPointCopy_CABLE_COUNT", "Polyline_CABLE_COUNT"),
    ("InputPointCopy_BUFFER_QTY", "Polyline_BUFFER_QTY"),
    ("InputPointCopy_FIBER_QTY", "Polyline_FIBER_QTY"),
    ("InputPointCopy_APPROVED_IND", "Polyline_APPROVED_IND"),
    ("InputPointCopy_CONDUIT_DIAMETER", "Polyline_CONDUIT_DIAMETER"),
    ("InputPointCopy_CONDUIT_MATERIAL", "Polyline_CONDUIT_MATERIAL"),
    ("InputPointCopy_CONDUIT_SEQUENTIAL_LENGTH", "Polyline_CONDUIT_SEQUENTIAL_LENGTH"),
    ("InputPointCopy_CONDUIT_OWNER", "Polyline_CONDUIT_OWNER"),
    ("InputPointCopy_INNERDUCT_QTY", "Polyline_INNERDUCT_QTY")
]

# Update cursor to iterate through features
with arcpy.da.UpdateCursor(line_copy, [pair[0] for pair in field_pairs] + [pair[1] for pair in field_pairs]) as cursor:
    for row in cursor:
        for i, pair in enumerate(field_pairs):
            row[i+len(field_pairs)] = row[i]  # Assign value from source field to target field
        cursor.updateRow(row)  # Update the row with the new values

# Save project
aprx.save()

arcpy.AddMessage("Field values copied successfully...")

#### - remove "Polyline_" from beginning of field names

In [ ]:
# Get a list of all field names in the feature class
field_list = arcpy.ListFields(line_copy)
total_fields = len(field_list)

# Loop through the fields and update the names
for field in field_list:
    if field.name.startswith("Polyline_"):
        new_name = field.name.replace("Polyline_", "")
        arcpy.management.AlterField(line_copy, field.name, new_name)

arcpy.AddMessage('"Polyline_" removed from beginning of field names...')

#### - delete fields starting with "InputPointCopy_"

In [ ]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~ delete fields starting with "InputPointCopy_" ~~~~~~~~~~~~~~~~~~~~~~~~~#

fields_to_delete = []
for fields in field_list:
    if fields.name.startswith("InputPointCopy_"):
        fields_to_delete.append(fields.name)

if len(fields_to_delete) > 0:
    arcpy.management.DeleteField(line_copy, fields_to_delete)
    arcpy.AddMessage('Fields starting with "InputPointCopy_" deleted...')

#### - remove single spaces, then for records where the "CABLE_COUNT" field value is not null, replace instances of "\P" with a single space

In [ ]:
#~~~~~~~~~~~~~~~~~~~~ remove single spaces, then replace "\P" with a space in "CABLE_COUNT" field ~~~~~~~~~~~~~~~~~~#

with arcpy.da.UpdateCursor(line_copy, "CABLE_COUNT") as cursor:
    for row in cursor:
        if row[0] is not None: # Check if the value in the "CABLE_COUNT" field is not null
            row[0] = row[0].replace(" ", "") # Remove single spaces
            row[0] = row[0].replace("\\P", " ") # Replace "\P" with a single space
            cursor.updateRow(row) # Update the row with the modified value

arcpy.AddMessage('"CABLE_COUNT" field cleaned...')

#### - remove the join on the original "Polyline" FC and add "Polyline_join" FC to the TOC

In [ ]:
##~~~~~~~~~~~~~~~~~~~~~~~~~~~~ remove the join on "Polyline" FC ~~~~~~~~~~~~~~~~~~~~~~~~~##

arcpy.management.RemoveJoin(polyline_fc)
arcpy.AddMessage(f'Join removed from "{polyline_fc}" FC...')

##~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Add "Polyline_join" FC to TOC ~~~~~~~~~~~~~~~~~~~~~~~~~##

layer_path = arcpy.management.MakeFeatureLayer(line_copy, line_copy) # Create a layer object from the feature class

m.addLayer(layer_path.getOutput(0))

arcpy.AddMessage(f'"{line_copy}" FC added to TOC...')

### c) "Point" feature translation

#### - select for desired features in the "Point" FC (in this case, access points)

In [ ]:
##~~~~~~~~~~~ ACCESS POINTS ~~~~~~~~~~~##

APs = "Translated_AccessPoints"
APs_path = f"{scratch_gdb}\{APs}"

# Select access points from "Point" FC
arcpy.management.SelectLayerByAttribute(point_fc,
                                        "NEW_SELECTION",
                                        "(Layer = 'Peds Proposed' Or Layer = 'Vaults Proposed') And ACCESS_POINT_TYPE IS NOT NULL")

#### - if the selection is greater than 0, create an empty FC in the workspace GDB and append the selected features to it

In [ ]:
if int(arcpy.management.GetCount(point_fc).getOutput(0)) > 0:
    arcpy.AddMessage("Access points selected...")

    # create empty FC for translated access points
    geometry_type = "POINT"
    template = f"{schema}AccessPoint"

    arcpy.management.CreateFeatureclass(scratch_gdb,
                                        APs,
                                        geometry_type,
                                        template,
                                        has_m,
                                        has_z,
                                        spatial_ref)

    arcpy.AddMessage(f"'{APs}' FC created...")

    # append selected features to "Translated_AccessPoints" FC
    arcpy.management.Append(point_fc, APs_path, "NO_TEST")

    arcpy.AddMessage(f"Selected access points appended to '{APs}' FC!")

else:
    arcpy.AddMessage("No features met the selection criteria for access points...")

##### *** all newly created FC's use an empty TDS FC as a template so that they adopt the TDS schema ***

##### *** this process is then repeated for all other point features (fiber splitters, port connection rows, fiber splice enclosures, and poles) ***

### d) "Polyline" feature translation

#### - select for desired features in the "Polyline" FC (in this case, fiber cables)

In [ ]:
##~~~~~~~~~~~ FIBER CABLES ~~~~~~~~~~~##

FCs = "Translated_FiberCables"
FCs_path = f"{scratch_gdb}\{FCs}"

# Select fiber cables from "Polyline_join" FC
arcpy.management.SelectLayerByAttribute(line_copy,
                                        "NEW_SELECTION",
                                        "Entity = 'LWPolyline' And Layer = 'Fiber Cable' And FIBER_CABLE_TYPE IS NOT NULL")

#### - if the selection is greater than 0, create an empty FC in the workspace GDB, append the selected polylines to it, and run the densify GP tool on the new FC

In [ ]:
if int(arcpy.management.GetCount(line_copy).getOutput(0)) > 0:
    arcpy.AddMessage("Fiber cables selected...")

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~ create empty FC for translated fiber cables ~~~~~~~~~~~~~~~~~~~~~~~~~#
    geometry_type = "POLYLINE"
    template = f"{schema}FiberCable"

    arcpy.management.CreateFeatureclass(scratch_gdb,
                                        FCs,
                                        geometry_type,
                                        template,
                                        has_m,
                                        has_z,
                                        spatial_ref)

    arcpy.AddMessage(f"'{FCs}' FC created...")

    # append selected features to "Translated_FiberCables" FC
    arcpy.management.Append(line_copy, FCs_path, "NO_TEST")

    arcpy.AddMessage(f"Selected fiber cables appended to '{FCs}' FC!")

    # densify
    arcpy.edit.Densify(FCs_path, "DISTANCE", 10)

    arcpy.AddMessage(f"'{FCs}' densified...")

else:
    arcpy.AddMessage("No features met the selection criteria for fiber cables...")

##### *** all newly created FC's use an empty TDS FC as a template so that they adopt the TDS schema ***

##### *** this process is then repeated for all other polyline features (fiber drop cables and conduits) ***

### e) "Polygon" feature translation

#### - select for serving area in the "Polygon" FC

In [ ]:
##~~~~~~~~~~~ SERVING AREA ~~~~~~~~~~~##

SAs = "Translated_ServingArea"
SAs_path = f"{scratch_gdb}\{SAs}"

# Select serving area from "Polygon" FC
arcpy.management.SelectLayerByAttribute(polygon_fc,
                                        "NEW_SELECTION",
                                        r"(Layer LIKE '%DFN%' And Entity = 'LWPolyline') Or (Layer LIKE '%FDH%' And Entity = 'LWPolyline')")

#### - if the selection is greater than 0, create an empty FC in the workspace GDB and append the selected polygon to it

In [ ]:
if int(arcpy.management.GetCount(polygon_fc).getOutput(0)) > 0:
    arcpy.AddMessage("Serving area selected...")

    # create empty FC for translated serving area
    geometry_type = "POLYGON"
    template = f"{schema}ServingArea"
    polygon_fc = "Polygon"

    arcpy.management.CreateFeatureclass(scratch_gdb,
                                        SAs,
                                        geometry_type,
                                        template,
                                        has_m,
                                        has_z,
                                        spatial_ref)

    arcpy.AddMessage("Translated serving area FC created...")

    # append selected features to "Translated_Conduits" FC
    arcpy.management.Append(polygon_fc, SAs_path, "NO_TEST")

    arcpy.AddMessage(f"Selected serving area appended to '{SAs}' FC!")

else:
    arcpy.AddMessage("No features met the selection criteria for serving areas...")

##### *** all newly created FC's use an empty TDS FC as a template so that they adopt the TDS schema ***

### e) TOC/GDB clean-up

#### - delete "InputPointCopy" FC

In [ ]:
##~~~~~~~~~~~~~~~~~~~~ delete the "InputPointCopy" FC ~~~~~~~~~~~~~~~~~~~~~~~~##
arcpy.management.Delete(out_copy)

arcpy.AddMessage('"InputPointCopy" FC deleted...' )

#### - remove the imported CAD data and "Polyline_join" FC from the TOC

In [ ]:
##~~~~~~~~~~~~~~~~~~~~ remove CAD import data from TOC ~~~~~~~~~~~~~~~~~~~~~~~~##
remove = ["Point", "Polyline", "Polygon", "Polyline_join"]

for layer_name in remove:
    layer_obj = m.listLayers(layer_name)[0] # Find the layer by name
    m.removeLayer(layer_obj) # Remove the layer from the TOC

arcpy.AddMessage('"Point", "Polyline", "Polygon", and "Polyline_join" FCs removed from TOC...')

# 3) Append translated features to TDS feature service

### a) set workspace

##### You could hardcode this if you wanted... but since it's a script tool "arcpy.GetParameterAsText()" is used so that the user can specify the desired GDB

In [ ]:
# set workspace
scratch_gdb = arcpy.GetParameterAsText (0)

arcpy.env.workspace = scratch_gdb

### b) specify the name and path of the desired translated FC (in this case, access points)

In [ ]:
##~~~~~~~~~~~~~~~~~~~~ ACCESS POINTS ~~~~~~~~~~~~~~~~~~~~~~~~~~##

# Specify the name and path of the feature class
FC_name = "Translated_AccessPoints"
APs = f"{scratch_gdb}/{FC_name}"

### c) if the FC exists in the workspace GDB, append it to the corresponding TDS feature service

In [ ]:
if arcpy.Exists(APs):
    # Define the paths and variables
    ServiceName = "AccessPoint"
    inputs = APs
    target = f"{TDS_layer}/{ServiceName}"
    schema_type = "NO_TEST"

    # Append the data only if the feature class exists
    arcpy.management.Append(inputs, target, schema_type)
    arcpy.AddMessage(f'"{FC_name}" appended to {TDS_layer}/{ServiceName}...')

else:
    arcpy.AddMessage(f'"{FC_name}" does not exist in the workspace geodatabase...')

##### *** this process is then repeated for the remaining translated features (FSE's, PCR's, splitters, poles, conduits, fiber cable/drop cable, and SA's) ***

# 4) Set definition query according to current project WBS code

### a) access the current pro project, map frame, and access the layers in the TOC

In [ ]:
import arcpy

# access current pro project
aprx = arcpy.mp.ArcGISProject("CURRENT")

# access current map frame
m = aprx.activeMap

# access layers in TOC
lyrList = m.listLayers()

### b) make list of target layer names

In [ ]:
ADP_GIS_EDIT = ["ADP_GIS_EDIT\FiberSpliceEnclosure", "ADP_GIS_EDIT\AccessPoint", "ADP_GIS_EDIT\FiberSplitter", "ADP_GIS_EDIT\Pole",
                "ADP_GIS_EDIT\FiberCable", "ADP_GIS_EDIT\Conduit", "ADP_GIS_EDIT\ServingArea"]

### c) set desired WBS code to be used in loops

##### You could hardcode this if you wanted... but since it's a script tool "arcpy.GetParameterAsText()" is used so that the user can specify the desired WBS code

In [ ]:
wbs_code = arcpy.GetParameterAsText (0)

#~~~~~ Change to desired proj WBS code! ~~~~~#
WBS = f"TDS_WBS_CODE = '{wbs_code}'"

### d) change "PortConnection" table definition query

In [ ]:
for m in aprx.listMaps():
    for tbl in m.listTables():
        if tbl.longName == "ADP_GIS_EDIT\PortConnection":
            tbl.updateDefinitionQueries([]) # Change the definition query of target table
            tbl.definitionQuery = WBS
        else:
            tbl.updateDefinitionQueries([])

arcpy.AddMessage("Port connection table DQ changed...")

### e) change the definition query of layers in the "ADP_GIS_EDIT" list

In [ ]:
for l in lyrList:
    if l.isGroupLayer:
        lyrs = l.listLayers()
        for lyr in lyrs:
            if lyr.longName in ADP_GIS_EDIT:
                lyr.updateDefinitionQueries([]) # Change the definition query of target layers
                lyr.definitionQuery = WBS
            else:
                lyr.updateDefinitionQueries([])

arcpy.AddMessage("Feature class DQ's changed...")